# Docker Swarm with Blinkt!

## Init Docker Swarm on Cluster

Zunächst die IP-Adresse feststellen.

In [1]:
ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1

192.168.56.188


In [2]:
IP_LEADER=$(ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1)
echo $IP_LEADER

192.168.56.188


In [3]:
docker node ls || docker swarm init --advertise-addr ${IP_LEADER}

Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.
Swarm initialized: current node (qr8dfz4zko3veiz8jx4ro4s5m) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-6cbc4uw16htoyo9fbyh7coqt7mzbe0ywl8a0zxan3f6ddai828-2cf5ts0j4s3evvkeorq4urk3o 192.168.56.188:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



In [4]:
JOIN_TOKEN=$(docker swarm join-token -q worker)
echo $JOIN_TOKEN

SWMTKN-1-6cbc4uw16htoyo9fbyh7coqt7mzbe0ywl8a0zxan3f6ddai828-2cf5ts0j4s3evvkeorq4urk3o


## SSH einrichten

In [5]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

ls: Zugriff auf /home/pi/.ssh/id_rsa nicht möglich: Datei oder Verzeichnis nicht gefunden
localhost | SUCCESS | rc=0 >>



Den SSH Zugang vorbereiten. Zunächst aus `knwon_hosts` löschen. Dann wieder bekannt geben und den ssh-key übertragen.

In [8]:
ping -c 1 cluster-01.local | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1

192.168.56.189


In [9]:
for i in 01 02 03 04; do
    # Remove host key for ip
    CL_HOSTNAME=cluster-${i}
    CL_IP=$(ping -c 1 ${CL_HOSTNAME} | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1)
    echo ${CL_HOSTNAME}
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster FQDN
    ssh-keygen -R ${CL_HOSTNAME}.local
    # IP Cluster 
    ssh-keygen -R ${CL_IP}
    # Add ip into known_hosts
    ssh-keyscan -H ${CL_HOSTNAME}.local >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_HOSTNAME} >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_IP} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@${CL_HOSTNAME}.local
done

cluster-01
ssh-keygen: /home/pi/.ssh/known_hosts: No such file or directory
ssh-keygen: /home/pi/.ssh/known_hosts: No such file or directory
ssh-keygen: /home/pi/.ssh/known_hosts: No such file or directory
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 192.168.56.189 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 192.168.56.189 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 192.168.56.189 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed
/usr/bin/ssh-copy-id: INFO: 1 key(s) remain to be installed -- if you are prompted now it is to install the new keys

Number of key(s) added: 1

Now try logging into the ma

Alle Clients als Worker dem Swarm beitreten.

In [13]:
for i in 01 02 03 04; do
    CL_HOSTNAME=cluster-${i}.local
    echo $CL_HOSTNAME
    CL_IP=$(ping -c 1 ${CL_HOSTNAME} | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1)
    echo $CL_IP
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP_LEADER}:2377" || \
    ssh pi@cluster-${i} "docker swarm leave --force"
done

cluster-01.local
192.168.56.189
This node joined a swarm as a worker.
cluster-02.local
192.168.56.191
This node joined a swarm as a worker.
cluster-03.local
192.168.56.187
This node joined a swarm as a worker.
cluster-04.local
192.168.56.196
This node joined a swarm as a worker.


In [14]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
qr8dfz4zko3veiz8jx4ro4s5m *   cluster-00          Ready               Active              Leader
q0j7demf0h049jx8scowmq53w     cluster-01          Ready               Active              
l66u3n3vb4bmori8e0lujewtu     cluster-02          Ready               Active              
kz5jfahil7vuny9f7sb94ihwh     cluster-03          Ready               Active              
6dp3c6cxmd75cpa5dv4avcvqb     cluster-04          Ready               Active              


## Visualizer anschauen

Einen graphischen Output als Service starten.

In [18]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  alexellis2/visualizer-arm

gasw190c9sv6hys0w87i7i9r7

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
fy: Service converged to verify that tasks are stable... 

In [19]:
echo "http://${IP_LEADER}:8000"

http://192.168.56.188:8000


In [20]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


Auf allen nodes den Monitor Dienst installieren. Dieser überwacht, wie viele whoami Container laufen.

Grün: Ein whoami-Container wird gestartet.

Rot: Ein whoami-Container stoppt.

Gelb: Für die Version 1.1.0

Blau: Für die Version 1.2.0

In [21]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  stefanscherer/monitor:1.1.0

akwg3jmt3gqskrr2pwz319857
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


Es läuft noch kein whoami.

In [22]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
akwg3jmt3gqs        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [23]:
docker service create --name whoami \
  --no-resolve-image \
  --detach=false \
  stefanscherer/whoami:1.1.0 

9hnen0vovuj01ohs7q218nxro

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
fy: Service converged to verify that tasks are stable... 

In [24]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
akwg3jmt3gqs        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
9hnen0vovuj0        whoami              replicated          1/1                 stefanscherer/whoami:1.1.0         


In [28]:
docker service scale --detach=false whoami=5

whoami scaled to 5

all progress: 0 out of 5 tasks 
   K$<3>$<3>
   K$<3>$<3>
   K$<3>$<3>
   K$<3>$<3>
all progress: 5 out of 5 tasks  
fy: Service converged to verify that tasks are stable... 

In [26]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
akwg3jmt3gqs        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
9hnen0vovuj0        whoami              replicated          5/5                 stefanscherer/whoami:1.1.0         


In [29]:
docker service scale --detach=false whoami=15

whoami scaled to 15

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
fy: Service converged to verify that tasks are stable... 

In [33]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
akwg3jmt3gqs        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
9hnen0vovuj0        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [30]:
docker service scale --detach=false whoami=40

whoami scaled to 40

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [32]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [ ]:
docker service ls

In [34]:
docker service scale --detach=false whoami=1

whoami scaled to 1

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks  
fy: Service converged to verify that tasks are stable... 

In [35]:
docker service scale --detach=false whoami=40

whoami scaled to 40

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [36]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
akwg3jmt3gqs        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
9hnen0vovuj0        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [37]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.1.0 whoami

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [38]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [39]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
akwg3jmt3gqs        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
9hnen0vovuj0        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [ ]:
END=10
for i in $(seq 1 $END); do
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    docker service scale --detach=false whoami=1 
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=15
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=40
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
done

whoami scaled to 1
Since --detach=false was not specified, tasks will be scaled in the background.
In a future release, --detach=false will become the default.
whoami

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
whoami scaled to 15ed to verify that tasks are stable... 
Since --detach=false was not specified, tasks will be scaled in the background.
In a future release, --detach=false will become the default.
whoami

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoami scaled to 40ed to verify that tasks are stable... 
Since --detach=false was not specified, tasks will be scaled in the background.
In a future release, --detach=false will become the default.
whoami

all progress: 40 out of 40 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress:

# Aufräumen

In [1]:
docker service rm whoami

whoami


In [2]:
docker service rm monitor

monitor


In [3]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
qr8dfz4zko3veiz8jx4ro4s5m *   cluster-00          Ready               Active              Leader
q0j7demf0h049jx8scowmq53w     cluster-01          Ready               Active              
l66u3n3vb4bmori8e0lujewtu     cluster-02          Ready               Active              
kz5jfahil7vuny9f7sb94ihwh     cluster-03          Ready               Active              
6dp3c6cxmd75cpa5dv4avcvqb     cluster-04          Ready               Active              


In [4]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
gasw190c9sv6        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [5]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm leave --force"
done

Node left the swarm.
Node left the swarm.
Node left the swarm.
Node left the swarm.


In [6]:
docker service rm visualizer

visualizer


In [7]:
docker swarm leave --force

Node left the swarm.
